# Purpose of this notebook
This notebook starts an instance of Neuroglancer using Python. This allows you to keep Neuroglancer open for as long as the notebook is open. It also allows you to control Neuroglancer using Python -- including which layers are loaded, their configuration and which annotations are displayed. We will use this to load in annotated cells from ClearMap2  from a pre-defined subvolume.


# Setup
In order to run the code in this notebook, you will need a conda environment with python3 and containing some additional libraries. This environment "ng" can be set up in the following way:
In terminal:
- conda create -n ng python=3.8 -y
- conda activate ng # (or source activate ng, depending on which version of conda you have)
- pip install cloud-volume
- pip install neuroglancer <br>

\# To enable you to use jupyter notebooks to work with this environment as a kernel:
In terminal:
- conda activate ng
- pip install --user ipykernel
- python -m ipykernel install --user --name=ng

Once this is all installed, make sure to select this conda environment as the kernel when running this notebook (you might have to restart the jupyter notebook server)

# Step 1: Host the light-sheet data on your machine so that Neuroglancer can see it
Open up a Python or iPython session outside of this jupyter notebook and run the following code. Make sure to have the ng conda environment activated before running python, otherwise you will get some module not found errors. You don't want to run this code in this notebook because it will cause it to hang and you won't be able to run the rest of the cells.

```python
from cloudvolume import CloudVolume
layer_dir = "/jukebox/LightSheetData/lightserv/cz15/zimmerman_01/zimmerman_01-001/imaging_request_1/viz/rawdata/blendeddata_zimmerman_01-001"
vol = CloudVolume(f'file://{layer_dir}')
vol.viewer(port=1338)
```

# Step 2: View your custom volume in Neuroglancer
Step 1 hosts your data via http on port 1338 of your local machine. To actually view your data in Neuroglancer, there are two ways to do this. You can either load the data in manually in the browser or load it in with python. 

For the python method (preferred), you can do this by executing the cells below. Make sure you have hosted the data in another python instance as in step1 before proceeding.

In [24]:
import neuroglancer
import numpy as np

In [14]:
# Set which client you want to use - use the BRAINCOGS client to get the latest features.
neuroglancer.set_static_content_source(url='https://neuroglancer-braincogs.appspot.com')

In [17]:
# Make a viewer object that represents your connection to a Neuroglancer session
viewer = neuroglancer.Viewer()

In [18]:
# Load in the light-sheet data layer with name "channel_647" or call it whatever you want. 
# This is the name for the layer that appears in the box in the upper left of the screen
# when you load in the layer. 
layer_name = "channel_647"
# This cell generates a link, which when clicked brings you to the
# Neuroglancer browser interface with your data loaded in 
with viewer.txn() as s:
    s.layers[layer_name] = neuroglancer.ImageLayer(
        source='precomputed://http://localhost:1338',
    )
print(viewer)

http://127.0.0.1:42971/v/9cd83e3d6102ab4837dfb8700ba3a4b1ded832db/


Click that link and it will bring you to Neuroglancer with your data loaded in.

You might notice that the contrast is poor. To change that execute this cell:

In [21]:
# Change image contrast so the data are more visible
# This inverts the color map (the 1.0- part) and increases the contrast (the factor of 300). 
# Use the d and f keys in the browser to update the contrast to your liking
with viewer.txn() as s:
    imagelayer = s.layers[layer_name]
    imagelayer.shader = """ void main() {emitGrayscale(1.0-toNormalized(getDataValue())*300.0);} """

To manually label cells create an annotation layer and follow the instructions here: https://brainmaps.princeton.edu/2020/08/probe-detection-example-with-neuroglancer/ (starting around 1 minute in)

## Load ClearMap2 cells from a subvolume into Neuroglancer

In [28]:
# First load in entire clearmap2 cell detections into an array 
path_to_cells_file = "/jukebox/LightSheetData/lightserv/cz15/zimmerman_01/zimmerman_01-001/imaging_request_1/output/clearmap2_results/cells_raw.npy"
raw_cell_coordinates = np.load(path_to_cells_file)

In [29]:
# Make sure cells were loaded in -- there should be ~4.7 million
len(raw_cell_coordinates)

4743122

In [31]:
raw_cell_coordinates[0]

(3593, 4549, 10, 5, 98873048., 98873048.)

In [44]:
%%time
# Restrict based on a subvolume of your choice
# This cell can take a while (~30 seconds) since it is looping over all of the cells
x_min,x_max=2000,3000
y_min,y_max=3000,3500
z_min,z_max=1600,1800
x_mask = raw_cell_coordinates
subvol_coordinates = []
for (x,y,z,s,src,bkg) in raw_cell_coordinates:
    if x>x_min and x<x_max and y>y_min and y<y_max and z>z_min and z<z_max:
        subvol_coordinates.append([x,y,z])

CPU times: user 15.9 s, sys: 0 ns, total: 15.9 s
Wall time: 15.9 s


[[2042, 3194, 1601],
 [2157, 3154, 1601],
 [2277, 3440, 1601],
 [2400, 3119, 1601],
 [2426, 3068, 1601],
 [2438, 3068, 1601],
 [2458, 3218, 1601],
 [2471, 3234, 1601],
 [2500, 3071, 1601],
 [2510, 3287, 1601],
 [2538, 3155, 1601],
 [2565, 3128, 1601],
 [2704, 3372, 1601],
 [2716, 3411, 1601],
 [2721, 3474, 1601],
 [2744, 3371, 1601],
 [2779, 3302, 1601],
 [2812, 3284, 1601],
 [2830, 3214, 1601],
 [2831, 3250, 1601],
 [2890, 3181, 1601],
 [2898, 3112, 1601],
 [2911, 3062, 1601],
 [2963, 3186, 1601],
 [2970, 3207, 1601],
 [2978, 3100, 1601],
 [2993, 3427, 1601],
 [2122, 3285, 1602],
 [2135, 3133, 1602],
 [2393, 3270, 1602],
 [2479, 3189, 1602],
 [2505, 3247, 1602],
 [2544, 3326, 1602],
 [2561, 3438, 1602],
 [2670, 3312, 1602],
 [2695, 3304, 1602],
 [2718, 3396, 1602],
 [2726, 3290, 1602],
 [2747, 3302, 1602],
 [2779, 3394, 1602],
 [2805, 3438, 1602],
 [2825, 3296, 1602],
 [2846, 3367, 1602],
 [2857, 3391, 1602],
 [2896, 3410, 1602],
 [2926, 3149, 1602],
 [2929, 3174, 1602],
 [2939, 3415,

In [46]:
# This list now contains the coordinates in the subvolume only
len(subvol_coordinates)

5021

In [49]:
# Load them into Neuroglancer as a new annotation layer
counter=0
with viewer.txn() as s:
    s.layers['subvol_annotation']=neuroglancer.AnnotationLayer()
    annotations=s.layers['subvol_annotation'].annotations
    
    for (x,y,z) in subvol_coordinates:
        pt=neuroglancer.PointAnnotation(point=[x, y, z],id=f'point{counter}')
        annotations.append(pt)
        counter+=1